In [114]:
import re
import os 

from os import listdir
from os.path import isfile, join
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import io

from pathlib import Path
from tqdm.notebook import trange, tqdm
from joblib import Parallel, delayed
import re
from skimage import exposure
import h5py
import pandas as pd

In [115]:
p_dir = (Path().cwd().parents[0]).absolute()
data_dir = p_dir / 'data' 

In [117]:
donors = [
    "LN Donor A",
    "LN Donor E",
    "INT Donor B",
    "INT Donor E",
    "TS Donor A",
    "TS Donor E",
    "SP Donor A"
]

donor = donors[3]

# Import data

In [118]:
h5_data = p_dir / 'data' / 'h5' / f'{donor}.hdf5'

def get_imgs(file_path, name):
    f = h5py.File(file_path, 'r')
    imgs = f[name]
    labels = list(f[name].attrs['labels'])
    return imgs, labels

def contrast_stretching(img):
    # Contrast stretching
    p2, p98 = np.percentile(img, (1, 95))
    img_rescale = exposure.rescale_intensity(
        img, in_range=(p2, p98), out_range=(0, 255)
    ).astype(np.uint8)
    return img_rescale


In [119]:
df = pd.read_csv(data_dir / 'metadata' / f"info_{donor}.csv")
ROIs = df.ROI.unique()

In [120]:
for roi in ROIs:
    imgs, markers = get_imgs(h5_data, str(roi))
    break

In [121]:
import napari 
imgs, markers = get_imgs(h5_data, str(1))
viewer = napari.view_image(np.stack(imgs), channel_axis=0, name=markers, visible=False)

In [126]:
imgs_dapi = []

# subset = ["DNA1", "DNA2", 'EZH2', 'Ki67', 'C-Myc']
subset = ["DNA1", "DNA2", 'Ki67', 'H3K27me3', 'C-Myc']
# subset = ["DNA1", "DNA2", 'Ki67', 'H3K27me3']
# subset = ["DNA1", "DNA2"]

for roi in ROIs:
    imgs, markers = get_imgs(h5_data, str(roi))
    img_dapi = np.max(
        contrast_stretching(imgs[sorted([markers.index(marker) for marker in subset])]), axis=0
    )
    img_dapi = contrast_stretching(img_dapi)
    imgs_dapi.append(img_dapi)

In [127]:
import napari

napari.view_image(np.stack(imgs_dapi))

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 499.5, 499.5), zoom=0.5073, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 16.0, 1.0), (0.0, 1000.0, 1.0), (0.0, 1000.0, 1.0)), current_step=(8, 500, 500), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x214ff36e700>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ]), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertices=Fa

In [128]:
nuclei_image_path = p_dir / 'data' / 'ROI_nuclei' /  f'{donor}'
nuclei_image_path.mkdir(parents=True, exist_ok=True)


In [129]:
for i, img in enumerate(imgs_dapi):
    io.imsave(nuclei_image_path / f'{ROIs[i]}.png', img)